In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "movie": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "영화 {movie}에 대해서 이모티콘을 세 개를 골라주세요."),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 질문한 영화를 나타내는 세 개의 이모티콘을 응답하는 AI입니다.",
        ),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("타이타닉")

{'question': '타이타닉'}
content='🚢💔🎶'


In [3]:
invoke_chain("명량")

{'question': '명량'}
content='⚓️🌊🔥'


In [4]:
invoke_chain("내가 바로 전에 물어본 2개의 영화는 무엇인가요?")

{'question': '내가 바로 전에 물어본 2개의 영화는 무엇인가요?'}
content='당신이 바로 전에 물어본 영화는 "대부"와 "타이타닉"입니다.'
